https://python.langchain.com/docs/use_cases/chatbots/quickstart

The chatbot interface is based around messages rather than raw text, and is best suited to Chat Models rather than text completion LLMs.

Generally the difference between Models and ChatModels, PromptTemplates and ChatPromptTemplates, Prompts and Messages, is that the formers are designed for string input->outputs, whereas the latters are designed for lists of conversations.

In [ ]:
%pip install --upgrade langchain langchain-openai typing-inspect typing_extensions pydantic

## A Basic ChatBot

In [1]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [3]:
chat.invoke("Translate this sentence from English to French: I love programming.")

AIMessage(content="J'adore la programmation.")

This input prompt can be wrapped more properly to differentiated better from AI messages:

In [6]:
from langchain_core.messages import HumanMessage
chat.invoke([HumanMessage("Translate this sentence from English to French: I love programming.")])

AIMessage(content="J'adore la programmation.")

The model on its own does not have any history or state, so if you ask:

In [7]:
chat.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content='I said, "What did you just say?"')

It doesn’t take the previous conversation turn into context, and cannot answer the question.

To get around this, **we need to pass the entire conversation history** into the model:

In [11]:
from langchain_core.messages import AIMessage

In [9]:
chat.invoke(
    [
        HumanMessage("Translate this sentence from English to French: I love programming."),
        AIMessage(content="J'adore la programmation."),
        HumanMessage("What did you just say?")
    ]
)

AIMessage(content='I said "J\'adore la programmation" which means "I love programming" in French.')

In [12]:
chat.invoke(
    [
        HumanMessage("Translate this sentence from English to French: I love programming."),
        AIMessage("J'adore la programmation."),
        HumanMessage("What did you just say?")
    ]
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.')

### Prompt Templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

In the above definition of `prompt`, `MessagesPlaceholder(variable_name="messages")` is going to be responsible for taking in the entire set of input messages all at once.

In [15]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.')

### Message History

We can use an **in-memory**, demo message history called `ChatMessageHistory` for convenience.

In [16]:
from langchain.memory import ChatMessageHistory

In [17]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [18]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."')

In [19]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

AIMessage(content='I said, "J\'adore la programmation," which means "I love programming" in French.')

In [20]:
demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'),
 AIMessage(content='whats up?'),
 HumanMessage(content='Translate this sentence from English to French: I love programming.'),
 AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."'),
 HumanMessage(content='What did you just say?')]

This is a basic chatbot!!!